In [5]:
import cirq
import numpy as np
from math import pi
#from cirq.ops import gate_operation
from cirq.value import Duration

#libraries to plot histogram
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

#google
from cirq import circuits, ops, sim, study, protocols, optimizers
from cirq.google import convert_to_xmon_gates
from cirq import circuits, ops, sim, study, protocols
from cirq.google import xmon_device

from cirq.devices import GridQubit

shots=1000

In [3]:
class Xmon10Device(cirq.Device):

  def __init__(self):
      self.qubits = [GridQubit(i, 0) for i in range(10)]

  def duration_of(self, operation):
      # Wouldn't it be nice if everything took 10ns?
      return cirq.Duration(nanos=10)

  def validate_operation(self, operation):
      if not isinstance(operation, cirq.GateOperation):
          raise ValueError('{!r} is not a supported operation'.format(operation))
      if not isinstance(operation.gate, (cirq.CZPowGate,
                                         cirq.XPowGate,
                                         cirq.PhasedXPowGate,
                                         cirq.YPowGate)):
          raise ValueError('{!r} is not a supported gate'.format(operation.gate))
      if len(operation.qubits) == 2:
          p, q = operation.qubits
          if not p.is_adjacent(q):
            raise ValueError('Non-local interaction: {}'.format(repr(operation)))

  def validate_scheduled_operation(self, schedule, scheduled_operation):
      self.validate_operation(scheduled_operation.operation)

  def validate_circuit(self, circuit):
      for moment in circuit:
          for operation in moment.operations:
              self.validate_operation(operation)

  def validate_schedule(self, schedule):
      for scheduled_operation in schedule.scheduled_operations:
          self.validate_scheduled_operation(schedule, scheduled_operation)

In [6]:
device = Xmon10Device()
circuit = cirq.Circuit()

In [7]:
circuit = cirq.Circuit()
circuit.append([cirq.CZ(device.qubits[0], device.qubits[1]), cirq.X(device.qubits[0])])
print(circuit)

(0, 0): ───@───X───
           │
(1, 0): ───@───────


In [8]:
schedule = cirq.moment_by_moment_schedule(device, circuit)

In [17]:
print(schedule[cirq.Timestamp(nanos=5)]) #the key is the star time of the Sorted Operation

[ScheduledOperation(cirq.Timestamp(picos=0), cirq.Duration(picos=10000), cirq.CZ.on(cirq.GridQubit(0, 0), cirq.GridQubit(1, 0)))]


In [20]:
slice = schedule[cirq.Timestamp(nanos=5):cirq.Timestamp(nanos=15)] #a start and end time
slice_schedule = cirq.Schedule(device, slice)
print(slice_schedule == schedule)

True


In [21]:
"""Simulator performance test for a larger circuit."""
# inputs to sim_test
nqubits = 20        # number of qubits
depth = 10          # depth of circuit
nreps = 1           # number of repetitions
verbose=False       # verbose output (prints circuit)

# do the timing test
time = sim_test(nqubits, depth, nreps, verbose=verbose)

# display the output
print("\nIt took %0.2f seconds to run the circuit." % time)

NameError: name 'sim_test' is not defined